# Домашнее задание по рекомендательным системам

В данном домашнем задании вам предлагается реализовать User-based рекомендательную систему. Так же требуется реализовать несколько вспомогательных функций, шаблоны которых вы можете найти в `utils.py`.

Требования к выполнению задания:
- Реализация функции из `utils.py` засчитывается, только если пройдены все соответствующие тесты из `test.py`. Запуск тестов: <font color='red'>pytest test.py</font>. Для тестов вам потребуются библиотеки `numpy`, `scipy`, `pytest` и `hypothesis`.
- Плагиат запрещен. Если будет замечено, что часть задания списана, то 0 баллов ставится как списывающему, так и давшему списать.
- Если пользуетесь кодом из открытых источников, то указывайте ссылки, откуда взяли решение. Иначе такой код может быть воспринят как плагиат.
- При выполнении задания нельзя использовать библиотеку `scipy` и функцию `numpy.linalg.norm`

При запуске тестов могут появиться предупреждения: PearsonRConstantInputWarning и PearsonRNearConstantInputWarning. На них можно не обращать внимания.

Возможный максимум баллов за задание: 10 баллов <br>
Дедлайн: ??? <br>
Штраф: ??? - будет ли в курсе штраф? <br>
<br>
Для ускорения проверки, напишите здесь получившееся количество баллов: ...

## 1. Метрика сходства
<b>1.1. Реализация метрик (2 балла)</b>

Первое, с чем необходимо разобраться, при реализации User-based подхода, это с метрикой, с помощью которой будет решаться, насколько похожи пользователи. Вам предлагается реализовать 2 метрики: на основе евклидовой метрики и коэффициент корреляции Пирсона. Шаблоны для обоих функций можете найти в `utils.py`. Не забудьте проверить реализацию на тестах.

Евклидова метрика:
\begin{equation}
d(p,q)=\sqrt{(p_1-q_1)^2+(p_2-q_2)^2+\dots+(p_n-q_n)^2} = \sqrt{\sum_{k=1}^n (p_k-q_k)^2}
\end{equation}

В этом случае $d(p, q) \in [0, \infty)$, при этом если $d(p, q) \to 0$, то $sim(p, q) \to 1$. С учетом этого конечная формула будет выглядеть следующим образом:
\begin{equation}
sim(p, q) = \frac{1}{1 + d(p, q)}
\end{equation}
Так же в этой формуле не будет проблем с делением на 0.

Коэффициент корреляции Пирсона:
\begin{equation}
r_{xy} = \frac {\sum_{i=1}^{m} \left( x_i-\bar{x} \right)\left( y_i-\bar{y} \right)}{\sqrt{\sum_{i=1}^{m} \left( x_i-\bar{x} \right)^2 \sum_{i=1}^{m} \left( y_i-\bar{y} \right)^2}}
\end{equation}

<b>1.2. (1 балл)</b>

Рассмотрим пользователей $u$ и $v$. Им соотвествуют векторы $x_u$ и $x_v$(строки с индексами u и v в таблице R??), где $x_u[i] = r_{ui}$ и $x_v[i] = r_{vi}$. Из лекции известно, что похожесть между векторами $x_u$ и $x_v$ вычисляются только для тех индексов i, для которых существует и $r_{ui}$, и $r_{vi}$. То есть верно следуюющее:
\begin{equation}
sim(u, v) = sim(x_uI_{uv}, x_vI_{uv}),
\end{equation}
где $I_{uv} = [i | \exists r_{ui} \& \exists r_{vi}]$. При этом если $I_{uv} = \emptyset$, то $sim(u, v) \to -\infty$.

Реализуйте два новых метода, которые переиспользуют написанные вами `euclidean_distance` и `pearson_distance`, добавляющие условия на $x_u$ и $x_v$. Считается, что $x_u[i] = 0$, если $\nexists r_{ui}$. То же верно для $x_v$.

При реализации заданий можно как написать новые функции, так и использовать декораторы.

In [1]:
import numpy as np
from utils import euclidean_similarity, pearson_similarity

In [79]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
def euclidean_similarity_with_restrictions(x_u, x_v):
    mask = (x_u != 0)&(x_v != 0)
    if len(x_u[mask]):
        return euclidean_similarity(x_u[mask], x_v[mask])
    else:
        return float("-inf")

def pearson_similarity_restrictions(x_u, x_v):
    mask = (x_u != 0)&(x_v != 0)
    if len(x_u[mask]):
        return pearson_similarity(x_u[mask], x_v[mask])
    else:
        return 0

## 2. User-based method
<b>2.1. (3 балла)</b> 

Реализовать User-based подход, реализовав методы класса `UserBasedRecommendation`, основанного на использовании `NearestNeighbors`. В качестве метрики для нахождения похожих пользователей может быть использована как евклидова метрика, так и коэффициент корреляции Пирсона.

Не забывайте, что `NearestNeighbors` ищет минимум расстояния между элементами, поэтому логично в качестве метрики при инициализации `NearestNeighbors` использовать обратную метрике схожести. То есть такую, что когда $sim(u, v) \to 1$, то $d(u, v) \to 0$. Например: $d(u, v) = 1 - sim(u, v)$

In [47]:
from sklearn.neighbors import NearestNeighbors
from typing import Optional

from collections import Counter

def Pmetric_for_NN(x_u, x_v):
    return 1 - pearson_similarity_restrictions(x_u, x_v)

def Emetric_for_NN(x_u, x_v):
    return 1 - euclidean_similarity_with_restrictions(x_u, x_v)

class UserBasedRecommendation:
    def __init__(self, metric: str = 'euclidean', n_recommendations: int = 5, alpha: float = 0.8):
        """
        Args:
            metric: name of metric: ['euclidean', 'pearson']
            n_recommendations: number of recommendations. Also can be specified self.make_recommendation
            alpha: similarity threshold: if sim(u, v) > alpha then u and v are similar
        """
        # your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
        self.metric = Pmetric_for_NN if metric == "pearson" else Emetric_for_NN
        self.n_recommendations = n_recommendations
        self.alpha = alpha

    def fit(self, X: np.array):
        """
        Args:
            X: matrix N x M where X[u, i] = r_{ui} if r_{ui} exists else X[u, i] = 0
        """
        # your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
        self.model = NearestNeighbors(n_neighbors=self.n_recommendations,
                                      metric=self.metric)
        self.X = X
        self.model.fit(X)
        

    def __find_closest_users(self, user_id: int, n_closest_users: int):
        # your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
        distances_indexes = self.model.radius_neighbors(self.X[user_id].reshape(1, -1),
                                                        1 - self.alpha,
                                                        sort_results=True)
        return distances_indexes[1][0][-n_closest_users:]
    

    def make_recommendation(self, user_id: int, n_recommendations: Optional[int] = None):
        """
        Args:
            user_id: user id to whom you want to recommend
            n_recommendations: number of recommendations
        """
        # your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
        u0 = self.__find_closest_users(user_id, self.n_recommendations) # close_users
        if len(u0) > 0:
            p = np.zeros(self.X.shape[1])
            for item in range(self.X.shape[1]):
                for user in u0:
                    if X[user, item]:
                        p[item] += 1
                p[item] /= len(u0)
            return np.argsort(p)[-n_recommendations:]
        return []

<b>2.2. (1 балла)</b>

Приведите пример, для которого использование разных метрик будет давать разные рекомендации. Объясните свой пример.

In [ ]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
Тут снизу уже был написано ответ, это не мой

<b>Объяснение:</b> Для ясности пронумеруем товары с 0. Видно, что третьему пользователю так же как и второму больше понравился товар 1 и чуть меньше понравился товар 2, тогда как первому пользователю оба эти товары не сильно понравились, причем товар 1 ему понравился меньше, чем товар 2, то есть наблюдается обратная завизимость. Но все эти факторы никак не учитываются при использовании евклидовой метрики, поэтому в первом случае алгоритм посчитал, что первый и третий пользователь похожи только по тому, что они ставят оценки примерно в одном и том отрезке: {0, 1}, что является довольно странным предположением. А алгоритм, использующий коэффициент корреляции Пирсона учитывает эти факторы, поэтому находит третьему пользователю соседа со схожими интересами в виде второго пользователя. Отсюда получаем разные рекомендации.

## 3. Оценка качества
<b>3.1. (1 балл)</b>

Реализуйте Average Precision at k и Mean Average Precision at k. Шаблоны можете найти в `utils.py`.
\begin{align*}
AP@K = \frac{1}{m}\sum_{k=1}^K P(k)*rel(k), \\
MAP@K = \frac{1}{|U|}\sum_{u=1}^{|U|}(AP@K)_u
\end{align*}
где $P(k)$ - Precision at k, $rel(k) = 1$, если рекомендация релевантна, иначе $rel(k) = 0$.

## 4. Применение модели
<b>4.1. (2 балла)</b>

Выгрузите датасет `ratings_small.csv`: https://www.kaggle.com/rounakbanik/the-movies-dataset#ratings_small.csv

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
data = pd.read_csv('ratings_small.csv', index_col=False)
data.shape

(100004, 4)

In [6]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

(1, 671, 671)

In [7]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

(1, 163949, 9066)

Для простоты работы с данными, измените нумерацию пользователей и фильмов так, чтобы нумерация начиналась с 0 и шла непрерывно.

In [8]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
data.userId -= 1

In [9]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
data.movieId = lb_make.fit_transform(data.movieId)

In [10]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

(0, 670, 671)

In [11]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

(0, 9065, 9066)

Удалим для наиболее активных пользователей 5 оценок

In [12]:
active_users = data.userId.value_counts()[:10].index
test_data = pd.DataFrame([], columns=data.columns)
for user_id in active_users:
    _, test = train_test_split(data[data.userId == user_id], test_size=5, random_state=42)
    test_data = test_data.append(test, ignore_index=True)
    data = data[~((data.userId == user_id) & (data.movieId.isin(test.movieId.values)))]
data.shape, test_data.shape

((99954, 4), (50, 4))

Преобразуем данные в таблицу `X`, с которой может работать `UserBasedRecommendation`, где $X_{ui} = r_{ui}$, если пользователь $u$ поставил оценку фильму $i$, и $X_{ui} = 0$, если пользователь $u$ не проставил оценку фильму $i$.

Вам может пригодиться `csr_matrix`.

In [13]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
# как пригодится csr_matrix, если нельзя пользоваться scipy...
X = np.zeros((data.userId.max() + 1, data.movieId.max() + 1))
for user in data.userId.unique():
    userItems = data[data.userId == user].movieId
    for item in userItems:
        X[user, item] = data.loc[(data.userId == user)&(data.movieId == item)].rating

Для пользователей, у которых были удалены фильмы, найдите топ 100 фильмов, который должен посмотреть каждый из этих пользователей, используя `UserBasedRecommendation`. Не забудьте подобрать параметр alpha.

In [81]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
test1 = UserBasedRecommendation("pearson", 100, 0.4)
test1.fit(X)
res = []
for userId in test_data.userId.unique():
    res.append(test1.make_recommendation(userId, 100))

D:\python\Tinkoff\rec\utils.py:39: RuntimeWarning: invalid value encountered in double_scalars
  return numerator / denumerator
D:\python\Tinkoff\rec\utils.py:39: RuntimeWarning: invalid value encountered in double_scalars
  return numerator / denumerator
D:\python\Tinkoff\rec\utils.py:39: RuntimeWarning: invalid value encountered in double_scalars
  return numerator / denumerator
D:\python\Tinkoff\rec\utils.py:39: RuntimeWarning: invalid value encountered in double_scalars
  return numerator / denumerator
D:\python\Tinkoff\rec\utils.py:39: RuntimeWarning: invalid value encountered in double_scalars
  return numerator / denumerator
D:\python\Tinkoff\rec\utils.py:39: RuntimeWarning: invalid value encountered in double_scalars
  return numerator / denumerator


In [82]:
res

[array([ 741, 4686,    6,  419,   24,   10, 3869,  101,  455,  564,  122,
         483,  519,  880, 2147,  208, 1303, 1906,  196,  888,  173, 1131,
         152, 1480, 2407,  381,  441,  304,  642, 1024,  361,   16,  953,
          87,   18,  520, 1129,  262,  955,   58,  733,  282, 3854,   32,
          37,    5,  615, 2374,  951,  198,  695,  447,  322,  331,  225,
           9, 2062,  163,  403,  258,  341,   48,  268,  294,  529,  617,
         140, 2288,  184,  866,  144,  203,  100,  314,  383,  328,  966,
         644,  535,  132,  561,  522,    0,  232,   45,  344,   31,  406,
         309,  472,  521,  129,  527,  523,  427,  524,  284,  321,  525,
         266], dtype=int64),
 array([2161, 2147, 1315, 2069, 1976, 1906, 1573, 1574, 1615, 2893,  649,
         884,  619, 4686,  173, 3803, 3799, 4846,  263,  282,  331,  334,
        3367,  622,  406,  341,   46,  576, 4227,   24,  494,  615,  840,
         880,  203,  567, 1343, 6142,  561, 1028, 1251,  524, 6446,  970,
        1

Используя метрику `MAP@5`, `MAP@10` и `MAP@100`, определите, насколько эффективна user-based рекомендательная система для данной задачи.

In [54]:
from utils import mapk

Как можно улучшить работу модели?

<b>Ответ:</b> Например, написать другую модель...
Если матрица X будет сильно разряжена, то мы получим очень плохие результаты. Можно попробовать реализовать модель со скрытыми переменными (которая talent factor models)